In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

In [ ]:
# default_exp data.block

In [ ]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from torch.utils.data import Dataset
from torch import nn


import numpy as np
import gym
import time,sys

# Local modules

# Data Transforms
> Fastrl transforms for iterating through environments

In [ ]:
# export
class DQN(Module):
    def __init__(self):
        self.policy=nn.Sequential(
            nn.Linear(4,50),
            nn.ReLU(),
            nn.Linear(50,2)
        )
    
    def forward(self,x): 
        return torch.argmax(self.policy(x),dim=0)

In [ ]:
# export
class TestDatasetNoModule(Dataset):
    def __init__(self,device='cpu'):
        self.device=device
        
    def __len__(self): return 10
    def __getitem__(self,idx):
        print('starting')
        sys.stdout.flush()
        try:
            print('making env')
#             env=gym.make('CartPole-v1')
            return torch.rand(1,4).to(device=self.device)
        except Exception as e:
            print(e,'it crashed lol omg')
            sys.stdout.flush()
            return torch.rand(1,4).to(device=self.device)

In [ ]:
for x in DataLoader(TestDatasetNoModule(),num_workers=0):
    print(x)

starting
making env
tensor([[0.6898, 0.5690, 0.1330, 0.5667]])
starting
making env
tensor([[0.6602, 0.6063, 0.0303, 0.0557]])
starting
making env
tensor([[0.4614, 0.2993, 0.9056, 0.0197]])
starting
making env
tensor([[0.2339, 0.1553, 0.9100, 0.9060]])
starting
making env
tensor([[0.8206, 0.4922, 0.3069, 0.4053]])
starting
making env
tensor([[0.9035, 0.8759, 0.9231, 0.6580]])
starting
making env
tensor([[0.0231, 0.6843, 0.2327, 0.0793]])
starting
making env
tensor([[0.4821, 0.7613, 0.0940, 0.1260]])
starting
making env
tensor([[0.4183, 0.2653, 0.5770, 0.2688]])
starting
making env
tensor([[0.4273, 0.4283, 0.6365, 0.6518]])


In [ ]:
# export
class TestDataset(Dataset):
    def __init__(self,policy,device='cpu'):
        self.policy=policy
        self.device=device
        self.policy.to(device=self.device)
        
    def __len__(self): return 100
    def __getitem__(self,idx):
        try:
            env=gym.make('CartPole-v1')
            next_state=env.reset()
            print(id(env),' ')
            next_state, r, is_done, _=env.step(self.policy(Tensor(next_state).to(device=self.device)).cpu().numpy())
            if is_done:next_state=env.reset()
            return Tensor(next_state).to(device=self.device)
        except Exception as e:
            print(e)
            return Tensor(np.random.rand(0,4)).to(device=self.device)

In [ ]:
for x in DataLoader(TestDataset(DQN()),num_workers=0):
    print(x)

139977293054288  
tensor([ 0.0254,  0.1748,  0.0327, -0.3102])
139977293054544  
tensor([-0.0058,  0.2294, -0.0013, -0.2506])
139977293054544  
tensor([ 0.0202,  0.1461,  0.0452, -0.3103])
139977293057424  
tensor([ 0.0473,  0.2161, -0.0238, -0.3068])
139977294950864  
tensor([-0.0400,  0.1890,  0.0343, -0.2807])
139977293031248  
tensor([-0.0155,  0.2256, -0.0389, -0.2680])
139977294950864  
tensor([-0.0394,  0.2203,  0.0388, -0.2861])
139977294950864  
tensor([-0.0017,  0.2053, -0.0472, -0.3537])
139977294950864  
tensor([-0.0465,  0.1818, -0.0181, -0.2942])
139977294950864  
tensor([-0.0293,  0.2093,  0.0041, -0.2827])
139977294950864  
tensor([ 0.0074,  0.2075,  0.0071, -0.2739])
139977294950864  
tensor([ 0.0177,  0.2052,  0.0296, -0.2949])
139977294950864  
tensor([-0.0118,  0.2329,  0.0234, -0.2557])
139977294950864  
tensor([ 0.0498,  0.1947, -0.0036, -0.3013])
139977294950864  
tensor([ 0.0251,  0.1750,  0.0231, -0.3283])
139977293033424  
tensor([-0.0235,  0.2304,  0.0027, -0

In [ ]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_nbdev_extension.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/05_data.block.ipynb
